In [ ]:
## global_time 추가, OUPUT_STATION의 tuple 형태, INPUT_STATION의 tuple 형태
def CAStar(graph,AGV_LIST,INPUT_STATION,OUTPUT_STATION,reservation_table,global_time):
    agv_avail = getAvailAGV(AGV_LIST,battery_threshold)
    total_cost = 0
    while len(agv_avail)!=0 and jobQ.job_number!=0:
        job = jobQ.getJob()
        print(job)
#         print(job['station_dest'])
        agv_avail = getAvailAGV(AGV_LIST, battery_threshold)
        for agv in agv_avail:
            agv.calculate_bid(job,warehouse,INPUT_STATION[0].coordinates,OUTPUT_STATION[0].coordinates)
            agv.bid_list.append(agv.calculate_bid(job,warehouse,INPUT_STATION[0].coordinates,OUTPUT_STATION[0].coordinates))
        for agv_ in agv_avail:
#             print(agv_bid_list)
            agv_bid_sort = sorted(agv_.bid_list,key = lambda x:x[0], reverse = False)
#             print(agv_bid_sort)
        bid_compare_battery=[]
        for i in range(len(agv_bid_sort)):
            if agv_bid_sort[i][1]==agv_bid_sort[0][1]:
                bid_compare_battery.append((agv_bid_sort[i]))
        bid_compare_battery_sort = sorted(bid_compare_battery, key= lambda x:x[1].battery, reverse =True)

        reset_graph(graph)
         
        if job['inout']=='in':
            castar_path = findPathAstar(graph,bid_compare_battery_sort[0][1],bid_compare_battery_sort[0][1].current_location,INPUT_STATION[0].coordinates,reservation_table)+findPathAstar(graph,bid_compare_battery_sort[0][1],INPUT_STATION[0].coordinates,Rack_enterence[job['station_dest']],reservation_table)
        else:
            castar_path = findPathAstar(graph,bid_compare_battery_sort[0][1],bid_compare_battery_sort[0][1].current_location,Rack_enterence[job['station_dest']],reservation_table)+findPathAstar(graph,bid_compare_battery_sort[0][1],Rack_enterence[job['station_dest']],OUTPUT_STATION[0].coordinates,reservation_table)
        if castar_path == None:
            return -1
        for time,value in enumerate(castar_path):
#             reservation_table[hash(value.depth, value.id)] = True 
            reservation_table[(value.depth+global_time, value.id)] = True  
        target_agv = bid_compare_battery_sort[0][1]
        select_agv_list.append(target_agv)
        target_agv.get_job(job,castar_path)
        print(" target_agv : {}, target_agv.battery: {}, destination : {}, castar_path : {}, available:{}".format(target_agv,target_agv.battery,job['station_dest'],[i.coordinates for i in castar_path],target_agv.available))
#         ### path 찍어보기 
#         for move in castar_path:
#             print(move.coordinates)
#         print(reservation_table)
        for agv in AGV_LIST:
                agv.clear_bid_list(AGV_LIST)
        agv_avail = getAvailAGV(AGV_LIST,battery_threshold)
        for agv in AGV_LIST:
            total_cost += len(agv.walking_path)
            return total_cost

In [ ]:
### start, goal 고친 후 
### gloabl_time을 추가한 이후 
def findPathAstar(graph, agv,start_tuple,goal_tuple,reservation_table,global_time):
#     print("type(start_tuple)",type(start_tuple))
    if type(start_tuple)==tuple:
        start = graph[start_tuple[0]][start_tuple[1]]
    else:
#         print("여기서 문제 발생",start_tuple)
#         print("여기서 문제2 발생",start_tuple.coordinates)
        start = graph[start_tuple.coordinates[0]][start_tuple.coordinates[1]] 
    if type(goal_tuple)==tuple:
        target = graph[goal_tuple[0]][goal_tuple[1]]
    else:
        target = graph[goal_tuple.coordinates[0]][goal_tuple.coordinates[1]]
#     print(")(start_tuple",start_tuple)
#     print(")(start_tuple[0]",start_tuple[0])
#     print(")(start_tuple[1]",start_tuple[1])
#     start = graph[start_tuple[0]][start_tuple[1]]
# #     print("goal_tuple",goal_tuple)
# #     print("agv_goal",agv_goal)
# #     print("agv_goal[0]",agv_goal[0])
# #     print("agv_goal[1]",agv_goal[1])
#     target = graph[goal_tuple[0]][goal_tuple[1]]
# #     print(target)
# #     target = graph[agv_goal[0]][agv_goal[1]]
    
    start.g = 0
    start.h = manhattan_distance(start,target)
    start.f = start.h
    
    neighbours = [(0,1),(1,0),(-1,0),(0,-1),(0,0)]
    open_list = []
    closed_list = set()   
    
    # add start to open list
    heappush(open_list, start)
    reached_target_last_step = False

    while open_list:
        current = heappop(open_list)
        closed_list.add(current)
      
        if current.coordinates == target.coordinates:
            if not reached_target_last_step:
                reached_target_last_step = True
            else:
                path_so_far = extract_path(current) 
                # 원래꺼
#                 current_depth = current.depth + global_time
                #내가 수정한 것
#                 current.depth = current.depth + global_time
            
                ##global time 추가한 부분
                for time, value in enumerate(path_so_far):
                    reservation_table[(value.depth+global_time, value.id)] = True
            
                reset_f_val_graph(graph)
                current.came_from = None
#                 next_path = findPathAstar(graph, agv, goal_tuple, reservation_table)
#                 if next_path == None:
#                     return None
                print(path_so_far)
                return path_so_far
#                 return path_so_far + next_path[1:]
        else:
            reached_target_last_step = False
            
#         print("a")   
            
        for (i,j) in neighbours:
            x, y = (current.coordinates[0] + i, current.coordinates[1] + j)
            if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
                # neighbour coordinates are out of the graph
                continue
#             print("c") 
            neighbour = graph[x][y]
            neighbour = copy.deepcopy(neighbour)
            neighbour.depth = current.depth + 1

            if neighbour.type == NodeType.OBSTACLE:
                # neighbour is not traversable, an obstacle
                #print("neigbout %d depth %d has OBSTACLE" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in closed_list:
                # neighbour case has already been considered
                #print("neigbout %d depth %d has CLOSED" % (neighbour.id, neighbour.depth))
                continue

            if collisionWillOccur(reservation_table, current, neighbour):
                # can't move to neighbour since another agent is there
                #print("neigbout %d depth %d has colliiiishhh" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in open_list:
                # make neighbour refer to correct node object
                for x in open_list:
                    if x == neighbour:
                        neighbour = x
                        break

            if neighbour not in open_list or current.g + 1 < neighbour.g:
                if neighbour.coordinates == target.coordinates:
                    # waiting at target is preferred
                    neighbour.g = current.g + 0
                else:
                    neighbour.g = current.g + 1
                neighbour.h = manhattan_distance(neighbour, target)
                neighbour.f = neighbour.g + neighbour.h

                # set parent
                neighbour.came_from = current

                if neighbour not in open_list:
                    # add newly discovered node to open list.
                    heappush(open_list, neighbour)
#     print("a")

    print("IMPOSSIBLE PROBLEM")
        

In [ ]:
### start, goal 고친 후 
### gloabl_time을 추가한 이후 
def findPathAstar(graph, agv,start_tuple,goal_tuple,reservation_table,global_time):
#     print("type(start_tuple)",type(start_tuple))
    if type(start_tuple)==tuple:
        start = graph[start_tuple[0]][start_tuple[1]]
    else:
#         print("여기서 문제 발생",start_tuple)
#         print("여기서 문제2 발생",start_tuple.coordinates)
        start = graph[start_tuple.coordinates[0]][start_tuple.coordinates[1]] 
    if type(goal_tuple)==tuple:
        target = graph[goal_tuple[0]][goal_tuple[1]]
    else:
        target = graph[goal_tuple.coordinates[0]][goal_tuple.coordinates[1]]
#     print(")(start_tuple",start_tuple)
#     print(")(start_tuple[0]",start_tuple[0])
#     print(")(start_tuple[1]",start_tuple[1])
#     start = graph[start_tuple[0]][start_tuple[1]]
# #     print("goal_tuple",goal_tuple)
# #     print("agv_goal",agv_goal)
# #     print("agv_goal[0]",agv_goal[0])
# #     print("agv_goal[1]",agv_goal[1])
#     target = graph[goal_tuple[0]][goal_tuple[1]]
# #     print(target)
# #     target = graph[agv_goal[0]][agv_goal[1]]
    
    start.g = 0
    start.h = manhattan_distance(start,target)
    start.f = start.h
    
    neighbours = [(0,1),(1,0),(-1,0),(0,-1),(0,0)]
    open_list = []
    closed_list = set()   
    
    # add start to open list
    heappush(open_list, start)
    reached_target_last_step = False

    while open_list:
        current = heappop(open_list)
        ## 내가 추가한 부분 
        current.depth = current.depth + global_time
        closed_list.add(current)
      
        if current.coordinates == target.coordinates:
            if not reached_target_last_step:
                reached_target_last_step = True
            else:
                path_so_far = extract_path(current) 
                # 원래꺼
#                 current_depth = current.depth + global_time
                #내가 수정한 것
#                 current.depth = current.depth + global_time
            
                for time, value in enumerate(path_so_far):
                    print("@@@@ time : {}, value : {}".format(time+global_time,value.coordinates))
                    reservation_table[(value.depth, value.id)] = True
#                     reservation_table[(value.depth+global_time, value.id)] = True
                reset_f_val_graph(graph)
                current.came_from = None
#                 next_path = findPathAstar(graph, agv, goal_tuple, rehttp://localhost:8888/notebooks/CAstar_python/0808-01(%EC%A4%91%EC%9A%94)%20CAstar%20%2B%20global%20tick%20%EB%B6%80%EB%B6%84%20%EC%B6%94%EA%B0%80%20%EC%8B%9C%EB%8F%84.ipynb#servation_table)
#                 if next_path == None:
#                     return None


                ## 노드 찍어보는 것 
                for i in path_so_far:
                    print(i.coordinates)
                                        
                return path_so_far
#                 return path_so_far + next_path[1:]
        else:
            reached_target_last_step = False
            
#         print("a")   
            
        for (i,j) in neighbours:
            x, y = (current.coordinates[0] + i, current.coordinates[1] + j)
            if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
                # neighbour coordinates are out of the graph
                continue
#             print("c") 
            neighbour = graph[x][y]
            neighbour = copy.deepcopy(neighbour)
            ## 이부분에 tick 추가 (global)
#             neighbour.depth = current.depth +global_time+ 1
            neighbour.depth = current.depth + 1

            if neighbour.type == NodeType.OBSTACLE:
                # neighbour is not traversable, an obstacle
                #print("neigbout %d depth %d has OBSTACLE" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in closed_list:
                # neighbour case has already been considered
                #print("neigbout %d depth %d has CLOSED" % (neighbour.id, neighbour.depth))
                continue

            if collisionWillOccur(reservation_table, current, neighbour):
                # can't move to neighbour since another agent is there
                #print("neigbout %d depth %d has colliiiishhh" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in open_list:
                # make neighbour refer to correct node object
                for x in open_list:
                    if x == neighbour:
                        neighbour = x
                        break

            if neighbour not in open_list or current.g + 1 < neighbour.g:
                if neighbour.coordinates == target.coordinates:
                    # waiting at target is preferred
                    neighbour.g = current.g + 0
                else:
                    neighbour.g = current.g + 1
                neighbour.h = manhattan_distance(neighbour, target)
                neighbour.f = neighbour.g + neighbour.h

                # set parent
                neighbour.came_from = current

                if neighbour not in open_list:
                    # add newly discovered node to open list.
                    heappush(open_list, neighbour)
#     print("a")

    print("IMPOSSIBLE PROBLEM")
        

In [ ]:
#내가 생각하는 맞는것 
def collisionWillOccur(reservation_table, current, neighbour):
    if (current.depth + 1,neighbour.id) in reservation_table:
        # Moving to node collision
        return True
    if (current.depth + 1,current.id) in reservation_table and (current.depth,neighbour.id) in reservation_table:
        # Swap collision
        return True
    return False

In [1]:
a = [1,2,3]

In [6]:
for i in range(len(a)):
    if 3 in a:
        continue
    else:
        print('c')


In [7]:
for i in range(100):       # 0부터 99까지 증가하면서 100번 반복
    if i % 2 == 0:         # i를 2로 나누었을 때 나머지가 0면 짝수
        continue           # 아래 코드를 실행하지 않고 건너뜀
    print(i)

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99


In [ ]:
def calculate_bid(self,job,warehouse,INPUT_STATION_tuple,OUTPUT_STATION_tuple,reservation_table,global_time):

In [ ]:
def findPathAstar(graph,agv,start_tuple,goal_tuple,reservation_table,global_time):

In [ ]:
def book_reservation_table(path_so_far,global_time,reservation_table):
    for time, value in enumerate(path_so_far):
        reservation_table[(value.depth+global_time, value.id)] = True
    return reservation_table

In [ ]:
### start, goal 고친 후 
### gloabl_time을 추가한 이후 
## findPathAstar에 reservation까지 같이 들어 있음 
## findPathAstar에서 예약부분만 따로 떼어냄 

def findPathAstar(graph, agv,start_tuple,goal_tuple,reservation_table,global_time):
    global_time = global_time
    
    if type(start_tuple)==tuple:
        start = graph[start_tuple[0]][start_tuple[1]]
    else:
        start = graph[start_tuple.coordinates[0]][start_tuple.coordinates[1]] 
        
    if type(goal_tuple)==tuple:
        target = graph[goal_tuple[0]][goal_tuple[1]]
    else:
        target = graph[goal_tuple.coordinates[0]][goal_tuple.coordinates[1]]
    
    start.g = 0
    start.h = manhattan_distance(start,target)
    start.f = start.h
    
    neighbours = [(0,1),(1,0),(-1,0),(0,-1),(0,0)]
    open_list = []
    closed_list = set()   
    
    # add start to open list
    heappush(open_list, start)
    reached_target_last_step = False

    while open_list:
        current = heappop(open_list)
        current.depth = current.depth 
#         current.depth = current.depth + global_time
#         print("current : {} , current.depth : {}".format(current.coordinates,current.depth))
        ## 내가 추가한 부분 
        closed_list.add(current)
        if current.coordinates == target.coordinates:
            if not reached_target_last_step:
                reached_target_last_step = True
            else:
                path_so_far = extract_path(current) 
                # 원래꺼
#                 current_depth = current.depth + global_time
                #내가 수정한 것
#                 current.depth = current.depth + global_time
            
#                 for time, value in enumerate(path_so_far):
# #                     print("@@@@ time : {}, value : {}".format(time+global_time,value.coordinates))
# #                     print("value.depth : {}, value.id : {}".format(value.depth,value.id))
#                     reservation_table[(value.depth+global_time, value.id)] = True
# #                     reservation_table[(value.depth+global_time, value.id)] = True
                reset_f_val_graph(graph)
                current.came_from = None
#                 next_path = findPathAstar(graph, agv, goal_tuple, rehttp://localhost:8888/notebooks/CAstar_python/0808-01(%EC%A4%91%EC%9A%94)%20CAstar%20%2B%20global%20tick%20%EB%B6%80%EB%B6%84%20%EC%B6%94%EA%B0%80%20%EC%8B%9C%EB%8F%84.ipynb#servation_table)
#                 if next_path == None:
#                     return None
                
#                 for i in range(len(graph)):
#                     for j in range(len(graph[0])):
#                         graph[i][j].current = 0
    
                

                ## 노드 찍어보는 것 
#                 for i in path_so_far:
#                     print(i.coordinates)
                                        
                return path_so_far
#                 return path_so_far + next_path[1:]
                
        else:
            reached_target_last_step = False
#         print("a")   
            
        for (i,j) in neighbours:
            x, y = (current.coordinates[0] + i, current.coordinates[1] + j)
            if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
                # neighbour coordinates are out of the graph
                continue
#             print("c") 
            neighbour = graph[x][y]
            neighbour = copy.deepcopy(neighbour)
            ## 이부분에 tick 추가 (global)
#             neighbour.depth = current.depth +global_time+ 1
            neighbour.depth = current.depth + 1

            if neighbour.type == NodeType.OBSTACLE:
                # neighbour is not traversable, an obstacle
                #print("neigbout %d depth %d has OBSTACLE" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in closed_list:
                # neighbour case has already been considered
                #print("neigbout %d depth %d has CLOSED" % (neighbour.id, neighbour.depth))
                continue

            if collisionWillOccur(reservation_table, current, neighbour, global_time):
                # can't move to neighbour since another agent is there
                #print("neigbout %d depth %d has colliiiishhh" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in open_list:
                # make neighbour refer to correct node object
                for x in open_list:
                    if x == neighbour:
                        neighbour = x
                        break

            if neighbour not in open_list or current.g + 1 < neighbour.g:
                if neighbour.coordinates == target.coordinates:
                    # waiting at target is preferred
                    neighbour.g = current.g + 0
                else:
                    neighbour.g = current.g + 1
                neighbour.h = manhattan_distance(neighbour, target)
                neighbour.f = neighbour.g + neighbour.h

                # set parent
                neighbour.came_from = current

                if neighbour not in open_list:
                    # add newly discovered node to open list.
                    heappush(open_list, neighbour)
#     print("a")

    print("IMPOSSIBLE PROBLEM")
        

In [8]:
#실험1 같은 time 상황에서 제대로 동작한다. path1이 설정되면 path2는 알아서 피하는 경로 설정 -> CA* 
#global tick을 포함해서 CA*가 잘 작동함 
## 그러나 작업을 새로 받지 앟으면 reservation table에 현재 위치를 계속 예약하는 것이 되어있지 않다, 
# 새로운 작업을 받지 않으면 치워버려야 한다. 어디로 치워야 하나? 뒤로 치우는 것이 어떨까?
path1 = findPathAstar(graph, agv1, (9,8),(4,0),reservation_table,0)
book_reservation_table(path1,0,reservation_table)
print(reservation_table)
print("여기까지 path1")
path2 = findPathAstar(graph, agv2, (7,9),(3,0),reservation_table,1)
book_reservation_table(path2,1,reservation_table)
print(reservation_table)
print("여기까지 path2")
path3 = findPathAstar(graph, agv3, (9,9),(3,3),reservation_table,3)
book_reservation_table(path3,3,reservation_table)
print(reservation_table)

NameError: name 'findPathAstar' is not defined

In [ ]:
{(0, 89): True, (1, 88): True, (2, 87): True, (3, 86): True, (4, 85): True, (5, 84): True, (6, 83): True, (7, 82): True, (8, 81): True, (9, 80): True, (10, 70): True, (11, 60): True, (12, 50): True, (13, 40): True, (14, 30): True, (15, 20): True, (16, 10): True, (17, 0): True, (18, 1): True, (19, 2): True, (20, 3): True, (21, 4): True, (22, 4): True}
여기까지 path1
{(0, 89): True, (1, 88): True, (2, 87): True, (3, 86): True, (4, 85): True, (5, 84): True, (6, 83): True, (7, 82): True, (8, 81): True, (9, 80): True, (10, 70): True, (11, 60): True, (12, 50): True, (13, 40): True, (14, 30): True, (15, 20): True, (16, 10): True, (17, 0): True, (18, 1): True, (19, 2): True, (20, 3): True, (21, 4): True, (22, 4): True, (1, 97): True, (2, 96): True, (3, 95): True, (4, 94): True, (5, 94): True, (6, 84): True, (7, 83): True, (8, 82): True, (9, 81): True, (10, 80): True, (11, 70): True, (12, 60): True, (13, 50): True, (14, 40): True, (15, 30): True, (16, 20): True, (17, 10): True, (18, 0): True, (19, 1): True, (20, 2): True, (21, 3): True, (22, 3): True}
여기까지 path2
{(0, 89): True, (1, 88): True, (2, 87): True, (3, 86): True, (4, 85): True, (5, 84): True, (6, 83): True, (7, 82): True, (8, 81): True, (9, 80): True, (10, 70): True, (11, 60): True, (12, 50): True, (13, 40): True, (14, 30): True, (15, 20): True, (16, 10): True, (17, 0): True, (18, 1): True, (19, 2): True, (20, 3): True, (21, 4): True, (22, 4): True, (1, 97): True, (2, 96): True, (3, 95): True, (4, 94): True, (5, 94): True, (6, 84): True, (7, 83): True, (8, 82): True, (9, 81): True, (10, 80): True, (11, 70): True, (12, 60): True, (13, 50): True, (14, 40): True, (15, 30): True, (16, 20): True, (17, 10): True, (18, 0): True, (19, 1): True, (20, 2): True, (21, 3): True, (22, 3): True, (3, 99): True, (4, 98): True, (5, 97): True, (6, 96): True, (7, 86): True, (8, 85): True, (9, 84): True, (10, 83): True, (11, 82): True, (12, 81): True, (13, 80): True, (14, 70): True, (15, 60): True, (16, 50): True, (17, 40): True, (18, 30): True, (19, 20): True, (20, 10): True, (21, 0): True, (22, 1): True, (23, 2): True, (24, 3): True, (25, 13): True, (26, 23): True, (27, 33): True, (28, 33): True}

In [ ]:
def main():
    global number_of_tests
    cost_CA= []
    input_size_items = []
    input_size_orders = []
    select_agv_list = []
    charging_agvs_list = []
    moving_for_charge_agvs_list = []
#     number_of_agents = 20
    failed_count = 0
    for test_number in range(0,number_of_tests):
        print("starting test {}".format(test_number))
        order_input = len(total_job_information)
        graph, INPUT_STATION, OUTPUT_STATION = create_Astar_graph(warehouse)
        for running_time in range(run_time+1):
            now_charging_agv = getChargingAGV(AGV_LIST)
            if len(now_charging_agv)!=0:
                print('charging_agv',now_charging_agv)
            if len(now_charging_agv)!=0:
                for charging_agv in now_charging_agv:
                    charging_agv.charge_agv(charging_agvs_list)
            if len(moving_for_charge_agvs_list)!=0:
    #             print('-',moving_for_charge_agvs_list)
                for moving_agvs in moving_for_charge_agvs_list :
                    moving_agvs.move_one_tick_for_charge(charging_agvs_list,moving_for_charge_agvs_list)
                    print("running time : {}, moving agv for charging : {}, current location : {}, current battery : {}".format(running_time, moving_agvs,moving_agvs.current_location,moving_agvs.battery))
                    if moving_agvs.charging == True:
                        print('moving_for_charge_agvs_list:',moving_for_charge_agvs_list,'moving_agvs',moving_agvs,'moving_agvs battery:',moving_agvs.battery)
    #                     moving_for_charge_agvs_list.remove(moving_agvs)
                        charging_agvs_list.append(moving_agvs)
        #       #사용가능한 agv와 job이 있을때 
            if running_time in job_create_time:
                print("@@@@@@@@@@@@@@@@@",running_time)
                jobQ.insertJob(total_job_information.get('{}'.format(running_time))[0],total_job_information.get('{}'.format(running_time))[1][0])
            agv_avail = getAvailAGV(AGV_LIST,battery_threshold)
        
            while len(agv_avail)!=0 and jobQ.job_number!=0:
                job = jobQ.getJob()
                print(job)
                print("runningtime : {}, available_agv : {}".format(running_time,[agv.agv_id for agv in agv_avail]))
        #         print(job['station_dest'])
                #4# agv_avail이 제대로 안나옴  -> 제대로 나옴 24 tick에는 작업1이 이미 종료됨
                # 2가지 case 1) 함수 2)get_job함수 
                
                agv_avail = getAvailAGV(AGV_LIST, battery_threshold)
#                 print("1agv_availagv_avail",agv_avail)
            #1# , #2#
                for agv in agv_avail:
                    agv.calculate_bid(job,graph,INPUT_STATION[0].coordinates,OUTPUT_STATION[0].coordinates)
               
                #통신이라 가정 
                send_bid(agv_avail)
                
#                 #4# 각 agv의 bid_list가 잘 나오는지 확인
#                 for _ in agv_avail:
#                     print(_.bid_list)

                for agv_ in agv_avail:
#                     print("agv_bid_listagv_bid_list",agv_.bid_list)
                    agv_bid_sort = sorted(agv_.bid_list,key = lambda x:x[2], reverse = False)
#                     print("agv_bid_sort",agv_bid_sort)
                bid_compare_battery=[]
                for i in range(len(agv_bid_sort)):
                    if agv_bid_sort[i][1]==agv_bid_sort[0][1]:
                        bid_compare_battery.append((agv_bid_sort[i]))
#                         print("bid_compare_battery",bid_compare_battery)
                bid_compare_battery_sort = sorted(bid_compare_battery, key= lambda x:x[0].battery, reverse =True)
#                 print("bid_compare_battery_sortbid_compare_battery_sort",bid_compare_battery_sort)
#                 print("bid_compare_battery_sort",bid_compare_battery_sort)
                reset_graph(graph)
#                 print("bid_compare_battery_sort[0][0]",bid_compare_battery_sort[0][0])
#                 print("bid_compare_battery_sort[0][0].current_location",bid_compare_battery_sort[0][0].current_location)
#                 print("INPUT_STATION[0].coordinates",INPUT_STATION[0].coordinates)
#                 print("Rack_enterence[job['station_dest']]",Rack_enterence[job['station_dest']])
                reset_graph(graph)
                if job['inout']=='in':
                    castar_path = findPathAstar(graph,bid_compare_battery_sort[0][0],bid_compare_battery_sort[0][0].current_location,INPUT_STATION[0].coordinates,reservation_table)+findPathAstar(graph,bid_compare_battery_sort[0][0],INPUT_STATION[0].coordinates,Rack_enterence[job['station_dest']],reservation_table)
                else:
                    castar_path = findPathAstar(graph,bid_compare_battery_sort[0][0],bid_compare_battery_sort[0][0].current_location,Rack_enterence[job['station_dest']],reservation_table)+findPathAstar(graph,bid_compare_battery_sort[0][0].current_location,Rack_enterence[job['station_dest']],OUTPUT_STATION[0].coordinates,reservation_table)
                if castar_path == None:
                    return -1
                for time,value in enumerate(castar_path):
        #             reservation_table[hash(value.depth, value.id)] = True 
                    reservation_table[(value.depth+running_time, value.id)] = True  
                target_agv = bid_compare_battery_sort[0][0]
                select_agv_list.append(target_agv)
                target_agv.get_job(job,castar_path)
#                 print("target_agv.availabletarget_agv.available",target_agv,target_agv.available)
                print("D running_time : {}, target_agv : {}, target_agv.battery: {}, destination : {}, castar_path : {}, available:{}".format(running_time,target_agv.agv_id,target_agv.battery,job['station_dest'],[i.coordinates for i in castar_path],target_agv.available))
                
                for agv_ in agv_avail:
                    agv_.bid_list = []
                agv_avail = getAvailAGV(AGV_LIST, battery_threshold)
#             one_algorithm_failed = False 
#             print("INPUT_STATION.coordinates",INPUT_STATION[0].coordinates,"OUTPUT_STATION.coordinates",OUTPUT_STATION[0].coordinates)
            #@#
            # cost 계산부분
#             current_COST = CAStar(graph,AGV_LIST,INPUT_STATION,OUTPUT_STATION,reservation_table,running_time)
#             if current_COST ==-1:
#                 impossible_problems_CA +=1
#                 one_algorithm_failed = True
#             cost_CA.append(current_COST)


##################################################추가#######################################################################    

    #         print('C running_time : {} , agv_avail : {}'.format(running_time,agv_avail))
            for moving_agv in select_agv_list:
                moving_agv.move_one_tick()
                print("E time : {}, moving agv : {}, battery : {}, current location : {}".format(running_time,moving_agv.agv_id,moving_agv.battery,moving_agv.current_location.coordinates))
                if len(moving_agv.path)==0:
                    select_agv_list.remove(moving_agv)
                    moving_agv.available = True
                    if moving_agv.battery < battery_threshold:
                        moving_for_charge_agvs_list.append(moving_agv)
                        moving_agv.go_to_charge(moving_agv.select_charging_station(warehouse,CHARGING_STATION))
        
        number_of_tests = number_of_tests - failed_count   
        if number_of_tests <= 0:
            print("No simulations were successful")
            return
        print("cost_CA",cost_CA)
        avg_cost_CA = np.sum(cost_CA) / number_of_tests
        print("The average cost for CA: %.3f" % (avg_cost_CA))
        print("impossible problems encountered: CA: %d" % (impossible_problems_CA))
                


In [ ]:
def book_reservation_table(path_so_far,global_time,reservation_table):

In [47]:
rt = {(0, 93): True, (1, 83): True, (1, 82): True}

In [51]:
b = []
for i in rt.keys():
    if i[0]==1:
        b.append(i)
for _ in b:
    del rt[_]

In [52]:
print(rt)

{(0, 93): True}


In [ ]:
'950': ['in', [(7, 3)]]}

In [54]:
a = [1,2,3,4]
b =[k for k in a if k!=2 and k==3 ]
print(b)


[3]


In [46]:
a = [1,2,3,4]
b =[k for k in a if k!=2 ]
print(b)


[1, 3, 4]


In [ ]:
b =[k for k in a if i>=2]
print(b)

In [ ]:
class ChargingStation(object):
    
    def __init__(self,charging_station_id,charging_station_location,):
        self.charging_station_location = charging_station_location
        if self.charging_station_location in agv_first_location_list:
            self.free = False
        else:
            self.free = True
        self.reserved = False
        
    def charging_station_reservation(self):
        self.reserved = True
        
    def charging_station_free(self):
        self.free = True
        self.reserved = False

In [ ]:
def getAvail_ChargingStation(CHARING_STATION):
    return [charging_station for charging_station in CHARING_STATION if charging_station.free == True and charging_station.reserved=False]

In [ ]:
CHARGING_STATION_1.charging_station_location.coordinates

In [ ]:
class AGV(object):
    max_battery = 200
    CHARGING_STATION = [(2,9),(6,9)]    
    
    def __init__(self,agv_id,current_location,job_for_bid,current_tick=0,cost=0,bid_list = []):

In [58]:
for i in range(100):
    print('agv'+str(i))

agv0
agv1
agv2
agv3
agv4
agv5
agv6
agv7
agv8
agv9
agv10
agv11
agv12
agv13
agv14
agv15
agv16
agv17
agv18
agv19
agv20
agv21
agv22
agv23
agv24
agv25
agv26
agv27
agv28
agv29
agv30
agv31
agv32
agv33
agv34
agv35
agv36
agv37
agv38
agv39
agv40
agv41
agv42
agv43
agv44
agv45
agv46
agv47
agv48
agv49
agv50
agv51
agv52
agv53
agv54
agv55
agv56
agv57
agv58
agv59
agv60
agv61
agv62
agv63
agv64
agv65
agv66
agv67
agv68
agv69
agv70
agv71
agv72
agv73
agv74
agv75
agv76
agv77
agv78
agv79
agv80
agv81
agv82
agv83
agv84
agv85
agv86
agv87
agv88
agv89
agv90
agv91
agv92
agv93
agv94
agv95
agv96
agv97
agv98
agv99


In [ ]:
select_charging_station